In [1]:
import pandas as pd
from haversine import haversine
import json
import datetime
import os
import numpy as np
import zipcodes
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

os.getcwd()

'/home/jian/Projects/C21/TA/TA_of_4_stores'

In [2]:
df_4_stores=pd.DataFrame()
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"DOWNTOWN","Google_Address":"22 Cortlandt St, New York, NY 10007","Google_Coor":[[40.7104649,-74.0107144]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"REGO_PARK","Google_Address":"61-35 Junction Blvd, Rego Park, NY 11374","Google_Coor":[[40.7332205,-73.8631858]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"CROSS_COUNTY","Google_Address":"750 Central Park Ave, Yonkers, NY 10704","Google_Coor":[[40.9262461,-73.8553119]]}))
df_4_stores=df_4_stores.append(pd.DataFrame({"Store_Name":"PARAMUS","Google_Address":"200 Bergen Town Center, Paramus, NJ 07652","Google_Coor":[[40.9150771,-74.0579318]]}))

df_4_stores=df_4_stores.reset_index()
del df_4_stores['index']

In [3]:
df_4_stores

,Google_Address,Google_Coor,Store_Name
0,"22 Cortlandt St, New York, NY 10007","[40.7104649, -74.0107144]",DOWNTOWN
1,"61-35 Junction Blvd, Rego Park, NY 11374","[40.7332205, -73.8631858]",REGO_PARK
2,"750 Central Park Ave, Yonkers, NY 10704","[40.9262461, -73.8553119]",CROSS_COUNTY
3,"200 Bergen Town Center, Paramus, NJ 07652","[40.9150771, -74.0579318]",PARAMUS


In [4]:
df_store_zip=pd.DataFrame()
i_counter=1
df_4_stores['zip_list_5_miles']=np.nan
for index, row in df_4_stores.iterrows():
    store_name=row['Store_Name']
    store_center=row['Google_Coor']
    store_zip=row['Google_Address'][-5:]
    df_0=pd.DataFrame({"Store_Name":store_name,"zip_cd":store_zip,"distance_miles":"store_zip"},index=[i_counter])
    i_counter+=1
    df_store_zip=df_store_zip.append(df_0)
    zip_list_by_store=[store_zip]
    
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=5:
            df=pd.DataFrame({"Store_Name":store_name,"zip_cd":zip_cd,"distance_miles":dist},index=[i_counter])
            i_counter+=1
            df_store_zip=df_store_zip.append(df)
            zip_list_by_store=zip_list_by_store+[zip_cd]
            
    df_4_stores.loc[index,'zip_list_5_miles']=str(list(set(zip_list_by_store)))

In [6]:
df_4_stores['zip_list_5_miles']=df_4_stores['zip_list_5_miles'].apply(lambda x: eval(x))
df_4_stores['zip_count']=df_4_stores['zip_list_5_miles'].apply(lambda x: len(x))

df_4_stores=df_4_stores[['Store_Name','Google_Address','Google_Coor','zip_list_5_miles']]

In [7]:
df_store_zip=df_store_zip.reset_index()

def get_city_state_of_zip(zip_cd):
    
    try:
        city=zipcodes.matching(zip_cd)[0]['city']
        state=zipcodes.matching(zip_cd)[0]['state']
    except:
        city=np.nan
        state=np.nan
    return city, state

df_store_zip['city']=df_store_zip['zip_cd'].apply(lambda x: get_city_state_of_zip(x)[0])
df_store_zip['state']=df_store_zip['zip_cd'].apply(lambda x: get_city_state_of_zip(x)[1])

In [12]:
df_store_zip['city','state']=df_store_zip['zip_cd'].apply(lambda x: get_city_state_of_zip(x))
df_store_zip.head(4)

,index,Store_Name,distance_miles,zip_cd,city,state,"(city, state)"
0,1,DOWNTOWN,store_zip,10007,NEW YORK,NY,"(NEW YORK, NY)"
1,2,DOWNTOWN,4.3827,11225,BROOKLYN,NY,"(BROOKLYN, NY)"
2,3,DOWNTOWN,4.49784,11221,BROOKLYN,NY,"(BROOKLYN, NY)"
3,4,DOWNTOWN,4.75348,11220,BROOKLYN,NY,"(BROOKLYN, NY)"


In [9]:
tableau_data=df_4_stores[['Store_Name','Google_Coor']]
tableau_data['Latitude']=tableau_data['Google_Coor'].apply(lambda x: x[0])
tableau_data['Longitude']=tableau_data['Google_Coor'].apply(lambda x: x[1])
tableau_data=tableau_data.append(df_store_zip)
tableau_data.head(2)
del tableau_data['Google_Coor']

tableau_data['Latitude']=tableau_data['Latitude'].fillna(0)
tableau_data['Longitude']=tableau_data['Longitude'].fillna(0)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
writer=pd.ExcelWriter("/home/jian/Projects/C21/TA/TA_of_4_stores/C21_TA_5_miles_for_4_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_4_stores.to_excel(writer,"Four_stores_selected",index=False)
df_store_zip.to_excel(writer,"Zips_5_miles_by_store",index=False)
tableau_data.to_excel(writer,"Data_for_Tableau",index=False)
writer.save()

In [11]:
zipcodes.matching("10007")

[{'active': True,
  'city': 'NEW YORK',
  'country': 'US',
  'lat': 40.71,
  'long': -73.99,
  'state': 'NY',
  'world_region': 'NA',
  'zip_code': '10007',
  'zip_code_type': 'STANDARD'}]